In [1]:
import rioxarray
import xarray
from rasterio.enums import Resampling
import numba
import numpy as np

In [2]:
chunksize = 1000
truth = xarray.open_dataarray(
    "../data/test_sites/oahu/in-situ-DEM/Job751453_usace2013_oahu_lmsl_dem_000_001.tif",
    chunks=chunksize,
    masked=True,
).sel(band=1)
measured = xarray.open_dataarray(
    "../data/test_sites/oahu/kalman_updated.tif", chunks=chunksize, masked=True
).sel(band=1)

In [11]:
truth.rio.transform()

Affine(1.0, 0.0, 610459.0,
       0.0, -1.0, 2393595.0)

In [4]:
newmrguy = measured.rio.reproject_match(
    truth, resampling=Resampling.bilinear, masked=True
)

In [14]:
newmrguy.rio.

30000

In [6]:
truth_bounds = truth.rio.bounds()
measured_bounds = newmrguy.rio.bounds()

newmrguy.rio.to_raster("../data/test_sites/oahu/reprojx_xarray.tif", windowed=True)

In [4]:
# error = xarray.DataArray((newmrguy-truth))

In [22]:
def rmse(a, b):
    return np.square(np.subtract(a, b))


# np.mean()

In [23]:
squared_error = (
    xarray.apply_ufunc(rmse, truth, newmrguy, dask="parallelized").mean().compute()
)

In [24]:
print(squared_error.data**0.5)

5.32212133623187
